In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Mi pc no aguanta apriori :<, pruebo con mlxtend
from fim import *
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules, fpmax

In [7]:
df = pd.read_csv('./data/cleaned_online_retail.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [8]:
#A lo largo del tiempo vemos dos variaciones por cada mitad del año, por eso creo que separarlo en mitad tiene sentido sin complicar las cosas demas
# Creo que obtendremos mejores resultados porque estamos comparando dos periodos con cambios muy marcados por la etapa del a;o.
part1 = df[df['InvoiceDate'] < '2011-06-01']
part2 = df[df['InvoiceDate'] >= '2011-06-01']

In [24]:
def patternMining(df_oh, min_support, nd):
    freq = fpgrowth(df_oh, min_support=min_support, use_colnames=True)
    print("Itemsets frecuentes de la particion numero", nd)
    print(freq.head())
    rules = association_rules(freq, metric="lift", min_threshold=1)
    print("Reglas de asociacion de la particion numero", nd)
    print(rules[['support', 'confidence', 'lift']].head())
    maximal = fpmax(df_oh, min_support=min_support, use_colnames=True)
    print("Maximales de la particion numero", nd)
    print(maximal.head())
    return freq, rules, maximal

In [25]:
transactions1 = part1.groupby('InvoiceNo')['Description'].apply(list).tolist()
te = TransactionEncoder()
te_ary = te.fit(transactions1).transform(transactions1)
oneHot1 = pd.DataFrame(te_ary, columns=te.columns_)

transactions2 = part2.groupby('InvoiceNo')['Description'].apply(list).tolist()
te = TransactionEncoder()
te_ary = te.fit(transactions2).transform(transactions2)
oneHot2 = pd.DataFrame(te_ary, columns=te.columns_)


In [26]:
fre1, rules1, max1 = patternMining(oneHot1, 0.01, 1)
fre2, rules2, max2 = patternMining(oneHot2, 0.01, 2)

Itemsets frecuentes de la particion numero 1
    support                               itemsets
0  0.112946   (WHITE HANGING HEART T-LIGHT HOLDER)
1  0.022972  (KNITTED UNION FLAG HOT WATER BOTTLE)
2  0.020699       (CREAM CUPID HEARTS COAT HANGER)
3  0.019981                  (WHITE METAL LANTERN)
4  0.017229       (RED WOOLLY HOTTIE WHITE HEART.)
Reglas de asociacion de la particion numero 1
    support  confidence       lift
0  0.012443    0.110169   2.495384
1  0.012443    0.281843   2.495384
2  0.017349    0.496575  11.247633
3  0.017349    0.392954  11.247633
4  0.010529    0.093220   2.668273
Maximales de la particion numero 1
   support                            itemsets
0  0.01005   (UNION JACK FLAG PASSPORT COVER )
1  0.01005    (FELTCRAFT PRINCESS OLIVIA DOLL)
2  0.01005  (HANGING WOOD AND FELT BUTTERFLY )
3  0.01005     (12 PENCILS TALL TUBE WOODLAND)
4  0.01005        (RECYCLED ACAPULCO MAT PINK)
Itemsets frecuentes de la particion numero 2
    support                    